In [14]:
from ultralytics import YOLO
import cv2
import os
import csv
from time import time
import supervision as sv

class ObjectDetection:

    def __init__(self, image_directory, keyframe_interval):
        self.image_directory = image_directory
        self.keyframe_interval = keyframe_interval

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)

        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names

        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.5)
        self.keyframe_detected = False
        self.confidence_values = []

    def load_model(self):
        model = YOLO("best.pt")
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        return results

    def plot_bboxes(self, results, frame):
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
        )
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
                       for _, confidence, class_id, tracker_id
                       in detections]
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        return frame

    def __call__(self):
        save_directory = 'fishresult'
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        frame_number = 0

        for file_name in os.listdir(self.image_directory):
            if file_name.endswith('.jpg'):
                start_time = time()

                image_path = os.path.join(self.image_directory, file_name)
                frame = cv2.imread(image_path)

                if frame is not None:
                    if self.keyframe_detected:
                        self.keyframe_detected = False
                        results = self.predict(frame)
                        frame = self.plot_bboxes(results, frame)

                    end_time = time()
                    elapsed_time = end_time - start_time
                    if elapsed_time > 0:
                        fps = 1 / elapsed_time
                    else:
                        fps = float('inf')

                    cv2.putText(frame, f'FPS: {int(fps)}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    cv2.imshow('YOLOv8 Detection', frame)

                    save_path = os.path.join(save_directory, f'frame_{frame_number}.jpg')
                    cv2.imwrite(save_path, frame)
                    print(f'Saved frame {frame_number} as {save_path}')

                    results = self.predict(frame)
                    confidence = results[0].boxes.conf.cpu().numpy()
                    self.confidence_values.append(confidence)

                    frame_number += 1

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

        save_path = os.path.join(save_directory, 'confidence.csv')
        with open(save_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Frame Number', 'Confidence'])
            for i, confidence in enumerate(self.confidence_values):
                writer.writerow([i, confidence])

        cv2.destroyAllWindows()

detector = ObjectDetection(image_directory=r'C:\Users\USER\AI\ultralytics\detectingphotoshootv23\test\images', keyframe_interval=1)
detector()


Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Using Device:  cpu



0: 448x640 2 photoshoots, 164.4ms
Speed: 1.3ms preprocess, 164.4ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)



Saved frame 0 as fishresult\frame_0.jpg
Saved frame 1 as fishresult\frame_1.jpg


0: 448x640 2 photoshoots, 130.9ms
Speed: 1.0ms preprocess, 130.9ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 448x640 2 photoshoots, 101.2ms
Speed: 1.0ms preprocess, 101.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)



...

Saved frame 412 as fishresult\frame_412.jpg
Saved frame 413 as fishresult\frame_413.jpg


0: 480x640 (no detections), 110.7ms
Speed: 1.0ms preprocess, 110.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 114.8ms
Speed: 1.0ms preprocess, 114.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



Saved frame 414 as fishresult\frame_414.jpg
Saved frame 415 as fishresult\frame_415.jpg


0: 640x640 (no detections), 128.9ms
Speed: 1.0ms preprocess, 128.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


OverflowError: cannot convert float infinity to integer